In [18]:
import torch
import seqgen.seq_gen as g
import random
import matplotlib.pyplot as plt
import seaborn as sns
from seqgen.model import seq2seq_lstm
from seqgen.vocabulary import *
from seqgen.model import transformer
from seqgen.datasets.sequences import *

torch.autograd.set_detect_anomaly(True)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
if torch.cuda.device_count():
    device="cuda"
else:
    device="cpu"
print("Device", device)

Device cpu


In [20]:
lr=1e-2
num_layers=1
embedding_dim=32
batch_size=512
max_length=50
heads=8
dropout=0

In [21]:
vocab_in = Vocabulary(vocab_filename="seqgen/vocab_in.txt")
vocab_out = Vocabulary(vocab_filename="seqgen/vocab_out.txt")

dataset = SyntheticSequenceDataset(vocab_in, vocab_out, max_length, batch_size, continue_prob=0.99, device="cpu")

In [22]:
input_seqs, coordinates, target_seqs = dataset[0]
input_seqs.shape, coordinates.shape, target_seqs.shape

(torch.Size([512, 51]), torch.Size([512, 51, 4]), torch.Size([512, 51]))

In [23]:
print(input_seqs[0, :-1])
print(target_seqs[0, :-1])
print(target_seqs[0, 1:])

tensor([ 0,  6, 16,  6,  7, 11, 12,  3,  6,  8, 10,  6,  4, 13, 15,  5,  6,  3,
        10, 12,  8,  7,  4, 14, 13,  4,  9, 12, 16, 14,  8, 12, 14, 16,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])
tensor([ 0,  4, 16,  5, 18, 13, 12, 16, 18,  6, 16,  6, 16,  8,  6, 16,  6, 13,
        15,  8, 12, 16,  6, 15, 11, 15,  8, 15, 10, 16, 18,  3, 15, 17, 14, 14,
        16, 14,  4, 10, 15,  9, 15, 12,  7,  7, 12,  3,  4,  1])
tensor([ 4, 16,  5, 18, 13, 12, 16, 18,  6, 16,  6, 16,  8,  6, 16,  6, 13, 15,
         8, 12, 16,  6, 15, 11, 15,  8, 15, 10, 16, 18,  3, 15, 17, 14, 14, 16,
        14,  4, 10, 15,  9, 15, 12,  7,  7, 12,  3,  4,  1,  1])


In [7]:
def permutate_tokens(input_seq):
    # Get the first index where tensor has an SOS or EOS token
    sos_idx = list(input_seq).index(0)
    eos_idx = list(input_seq).index(1)
    # permutate all elements that are not SOS or EOS
    idx_permuted = torch.cat([torch.arange(0, sos_idx+1), (torch.randperm(eos_idx - sos_idx - 1) + sos_idx+1), torch.arange(eos_idx, max_length+1)])
    return idx_permuted

# The Transformer

In [8]:
load_from_checkpoint = False
checkpoint_file = "model_2023-01-15_09-17-53.pt"

# Transformer model
model = transformer.Transformer(
    src_vocab_size=len(vocab_in),
    trg_vocab_size=len(vocab_out),
    embedding_dim=embedding_dim,
    num_layers=num_layers,
    heads=heads,
    dropout=dropout,
    src_pad_idx=1e10,
    trg_pad_idx=1e10,
    device=device
).to(device)

# Initialize optimizer for encoder and decoder
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Load model weights from checkpoint
if load_from_checkpoint:
    checkpoint = torch.load(checkpoint_file)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [9]:
# Run the feature sequences through the model
output = model(input_seqs, target_seqs, coordinates)

In [10]:
# Get the predicted classes of the model
topv, topi = output.topk(1, dim=2)
output.shape, topi.shape, topv.shape

(torch.Size([512, 51, 25]), torch.Size([512, 51, 1]), torch.Size([512, 51, 1]))

In [11]:
loss = 0.0
for i in range(max_length):
    loss += criterion(output[:, i, :], target_seqs[:, i])
loss.item() / max_length

3.2522048950195312

# Training

In [ ]:
history = []
accuracies = []

for epoch in range(50000):    
    # Get a batch of training data
    input_seqs, coordinates, target_seqs = dataset[0]
    #positions_targets = seq2seq_lstm.get_position_encoding(max_length, embedding_dim, device=device).repeat(batch_size, 1, 1)
    
    # Set gradients of all model parameters to zero
    optimizer.zero_grad()

    # Initialize loss
    loss = 0
    accuracy = 0.0

    #####################
    #    TRANSFORMER    #
    #####################
    
    # Run the input sequences through the model
    output = model(input_seqs[:, :-1], target_seqs[:, :-1], coordinates[:, :-1])
    
    # Iterate over sequence positions to compute the loss
    for i in range(max_length-1):
        # Get the predicted classes of the model
        topv, topi = output[:, i, :].topk(1)
        loss += criterion(output[:, i, :], target_seqs[:, i+1])
        accuracy += float((topi.squeeze() == target_seqs[:, i+1]).sum() / (target_seqs.size(0)*(target_seqs.size(1)-2)))
    
    history.append(loss.item())
    accuracies.append(accuracy)
    
    print_every = 10
    if not epoch % print_every:
        _accuracy = sum(accuracies[-print_every:]) / print_every
        lr = scheduler.get_last_lr()[0]
        print(f"LOSS after epoch {epoch}", loss.item() / (target_seqs.size(1)), "LR", lr, "ACCURACY", _accuracy)

    # Compute gradient
    loss.backward()
    accuracy = 0.0

    # Update weights of encoder and decoder
    optimizer.step()

LOSS after epoch 0 5.782796223958333 LR 0.01 ACCURACY 0.03133370513096452
LOSS after epoch 10 2.0091868381874236 LR 0.01 ACCURACY 0.2857700885739177
LOSS after epoch 20 1.9814607208850337 LR 0.01 ACCURACY 0.30780054137576374
LOSS after epoch 30 1.8554617189893536 LR 0.01 ACCURACY 0.3315409750211984
LOSS after epoch 40 1.728349573471967 LR 0.01 ACCURACY 0.36774952001869676
LOSS after epoch 50 1.6991493374693627 LR 0.01 ACCURACY 0.40435267696157096
LOSS after epoch 60 1.5569861916934742 LR 0.01 ACCURACY 0.4239716183394194
LOSS after epoch 70 1.5916507197361367 LR 0.01 ACCURACY 0.42978714760392905
LOSS after epoch 80 1.482731538660386 LR 0.01 ACCURACY 0.4398317909799516
LOSS after epoch 90 1.4392969468060661 LR 0.01 ACCURACY 0.45207668729126455
LOSS after epoch 100 1.4778864243451286 LR 0.01 ACCURACY 0.451084181945771
LOSS after epoch 110 1.4708017087450214 LR 0.01 ACCURACY 0.4498764343559742
LOSS after epoch 120 1.4116333606196385 LR 0.01 ACCURACY 0.4600964598823339
LOSS after epoch 130 

#### Save model history

In [13]:
import pickle
from datetime import datetime

model_data = {
    "history": history,
    "lr": lr,
    "num_layers": num_layers,
    "embedding_dim": embedding_dim,
    "batch_size": batch_size,
    "max_length": max_length,
    "heads": heads,
    "dropout": dropout,
}

now = datetime.now() # current date and time
date_time = now.strftime("%Y-%m-%d_%H-%M-%S")

torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
    "history": history,
    "lr": lr,
    "embedding_dim": embedding_dim,
    "batch_size": batch_size,
    "max_length": max_length,
    "num_layers": num_layers,
    "heads": heads,
    "dropout": dropout,
}, "transformer_" + date_time + ".pt")


with open("training_" + date_time + '.pkl', 'wb') as f:
    pickle.dump(model_data, f)

## Make predictions

We run our input sequences through the model and get output seuences. Then we decode the output sequences with the Vocabulary class and get our final latex code.

In [ ]:
def predict(input_seqs, coordinates, target_seqs):
    vocab_in = Vocabulary(vocab_filename="seqgen/vocab_in.txt")
    vocab_out = Vocabulary(vocab_filename="seqgen/vocab_out.txt")

    with torch.no_grad():
        coordinate_encoding = seq2seq_lstm.get_coordinate_encoding(coordinates, d=embedding_dim, max_length=max_length)
        output = model(input_seqs, target_seqs, coordinate_encoding)
        # Get the predicted classes of the model
        topv, topi = output.topk(1, dim=2)
        
        return topi.squeeze()
    
def predict_sequentially(input_seqs, coordinates):
    prediction = torch.zeros((input_seqs.size(0), input_seqs.size(1)-1)).to(torch.int64)
    for i in range(max_length-1):
        output = predict(input_seqs, coordinates, prediction)
        prediction[:, i] = output[:, i]
    return prediction

In [ ]:
prediction = predict_sequentially(input_seqs, coordinates)
prediction.shape

In [ ]:
# Pick random sequence and its prediction from the model
import random

vocab_in = Vocabulary(vocab_filename="seqgen/vocab_in.txt")
vocab_out = Vocabulary(vocab_filename="seqgen/vocab_out.txt")

predictions = predict(input_seqs, coordinates, target_seqs)

i = random.randint(0, predictions.size(0)-1)
print("MODEL INPUT", vocab_in.decode_sequence(input_seqs[i, 1:].cpu().numpy()))
print("MODEL OUTPUT", vocab_out.decode_sequence(predictions[i, :-1].cpu().numpy()))
print("TARGET OUTPUT", vocab_out.decode_sequence(target_seqs[i, 1:].cpu().numpy()))

In [ ]:
prediction = vocab_out.decode_sequence(predictions[i].cpu().numpy())
prediction = list(filter(lambda x: x != '<end>', prediction))
prediction = "".join(prediction)
print("MODEL OUTPUT", prediction)

In [ ]:
predict_sequentially(input_seqs[0:3], coordinates[0:3])

In [ ]:
target_seqs[0:3, 1:]

## Prediction for permutated sequences

In [ ]:
def generate_permutated_batch(input_seq, coordinates):
    seqs = torch.zeros((5, input_seq.size(0))).to(torch.int64)
    coords = torch.zeros((5, coordinates.size(0), coordinates.size(1)))
    for i in range(5):
        idx_permutated = permutate_tokens(input_seq)
        seqs[i, :] = input_seq[idx_permutated]
        coords[i, :] = coordinates[idx_permutated]
    return seqs, coords

In [ ]:
input_permutated, coords_permutated = generate_permutated_batch(input_seqs[0], coordinates[0])
input_permutated

In [ ]:
predict_sequentially(input_permutated, coords_permutated)

In [ ]:
target_seqs[0, 1:]